In [2]:
# Place to dowload book free
#https://annas-archive.org/
import spacy
import ebooklib
from ebooklib import epub  
from html.parser import HTMLParser #(usar beutiful soup)
from collections import defaultdict
from spacy.lang.es import Spanish
from bs4 import BeautifulSoup
from lxml import etree
# En spacy, los saltos de linea los ignora, asi que supongo que para separar en parrafos o bien ingnorarlos, o bien procesarlos por separado,
# haciendo un split por \n



In [3]:
# paths and data acquisition 
book_path = ".\Asesinato en el Orient Express -- Agatha Christie -- Hércules Poirot 10, 1934 -- cba8798523cc82925c66887e83cdaa07 -- Anna’s Archive.epub"
book = epub.read_epub(book_path)


title = book.get_metadata('DC', 'title')[0][0]
print(f"The book {title} was correctly read")


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\pable\AppData\Local\Temp\ipykernel_9480\82667957.py:2: SyntaxWarning: invalid escape sequence '\A'
  book_path = ".\Asesinato en el Orient Express -- Agatha Christie -- Hércules Poirot 10, 1934 -- cba8798523cc82925c66887e83cdaa07 -- Anna’s Archive.epub"
c:\Users\pable\miniconda3\envs\main_env\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


The book Asesinato en el Orient Express was correctly read


c:\Users\pable\miniconda3\envs\main_env\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [4]:
# Function to return all the titles of the book

def get_book_titles(book):
    titles = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_body_content()

            # Step 1: Decode the byte string to a regular string
            decoded_str = content.decode('utf-8')

            # Step 2: Parse the decoded string with BeautifulSoup
            soup = BeautifulSoup(decoded_str, 'html.parser')

            # Step 3: The chapter titles are inside <h2> tag, Find the <h2> tag and get the 'title' attribute
            h2_tag = soup.find('h2')
            if h2_tag and 'title' in h2_tag.attrs:
                title_content = h2_tag['title']
                titles.append(title_content)

    return titles

titles = get_book_titles(book)
print(titles)


['1. El pasajero del «Taurus Express»', '2. El hotel Tokatlian', '3. Poirot renuncia a un caso', '4. Un grito en la noche', '5. El crimen', '6. ¿Una mujer?', '7. El cadáver', '8. El caso Armstrong', '1. Declaración del conductor del coche dormitorio', '2. Declaración del secretario', '3. Declaración del criado', '4. Declaración de la dama norteamericana', '5. Declaración de la dama sueca', '6. Declaración de la princesa rusa', '7. Declaración del conde y la condesa Andrenyi', '8. Declaración del coronel Arbuthnot', '9. Declaración de míster Hardman', '10. Declaración del italiano', '11. Declaración de miss Debenham', '12. Declaración de la doncella alemana', '13. Resumen de las declaraciones de los viajeros', '14. El arma', '15. Los equipajes', '1. ¿Cuál de ellos?', '2. Diez preguntas', '3. Algunos puntos sugestivos', '4. La mancha de grasa en un pasaporte húngaro', '5. El nombre de pila de la princesa Dragomiroff', '6. Una entrevista con el coronel Arbuthnot', '7. La identidad de Mary

In [ ]:
# Function to return all the text of the book until a chapter is found via the number of the chapter or the name of the chapter

def get_book_text_until_chapter(book,title_search_method = None):

    # First if user only knows te title of the chapter, we search for it in all the titles, to ensure it exists and get the index
    title_index = ''

    if isinstance(title_search_method,(int,float)):
        title_index = title_search_method

    elif isinstance(title_search_method,str):
        titles = get_book_titles(book)
        for i,title in enumerate(titles):
            if title_search_method in title:
                print(f"The chapter provided is the chapter {title_index}")
                title_index = i
                break
        
        if title_index is None:
            print(f"Title {title_search_method} not found")
            return None

    else:
        print("No title provided, please provide some title index of text")
        return None

    book_text = ''
    current_title_index = 0
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_body_content()
            decoded_str = content.decode('utf-8')

            soup = BeautifulSoup(decoded_str, 'html.parser')
            h2_tag = soup.find('h2')

            # Each chapter is saved inside a single ITEM_DOCUMENT
            # If the item is a title and different for the index or text we are looking for, add the text to the book_text var

            if h2_tag and 'title' in h2_tag.attrs:
                if current_title_index == title_index:
                    return book_text
                
                book_text += decoded_str
                current_title_index +=1


book_text = get_book_text_until_chapter(book,1)

print(book_text)

&#13;
  <h2 class="oculto" title="1. El pasajero del «Taurus Express»"/>&#13;
&#13;
  <div class="capitulo centrado">&#13;
    <p class="salto30">1</p>&#13;
&#13;
    <p class="salto05">EL PASAJERO DEL «TAURUS EXPRESS»</p>&#13;
  </div>&#13;
&#13;
  <p class="asangre salto35"><span class="letra">E</span>RAN las cinco de una madrugada de invierno en Siria. Junto al andén de Alepo estaba detenido el tren que las guías de ferrocarriles designan con el nombre de <i>Taurus Express</i>. Estaba formado por un coche con cocina comedor, un coche cama y dos coches corrientes.</p>&#13;
&#13;
  <p>Junto al estribo del coche cama se encontraba un joven teniente francés, de resplandeciente uniforme, conversando con un hombrecillo embozado hasta las orejas, del que sólo podían verse la punta de la nariz y las dos guías de un enhiesto bigote.</p>&#13;
&#13;
  <p>Hacía un frío intensísimo, y aquella misión de despedir a un distinguido forastero no era cosa de envidiar, pero el teniente Dubosc la cumplí

In [ ]:
def get_chapter_text(epub_path, chapter_title=None, chapter_index=None):

    # Load the EPUB book
    book = epub.read_epub(epub_path)
    
    # Initialize an empty string for the chapter text
    chapter_text = ""
    
    # Check for title match or use index
    for i, item in enumerate(book.get_items_of_type(ebooklib.ITEM_DOCUMENT)):
        print([name for name in item.get_name()])
    #     # Check if using title match or specific index
    #     if chapter_title:
    #         if chapter_title in item.get_name():
    #             soup = BeautifulSoup(item.get_body_content(), 'html.parser')
    #             chapter_text = soup.get_text(separator="\n")  # Extract text with line breaks
    #             break
    #     elif chapter_index is not None:
    #         if i == chapter_index:
    #             soup = BeautifulSoup(item.get_body_content(), 'html.parser')
    #             chapter_text = soup.get_text(separator="\n")
    #             break
    
    # if not chapter_text:
    #     print("Chapter not found.")
    # return chapter_text


In [ ]:
# class to store the chapters
class Chapter():
    def __init__(self,chapter_name,chapter_text) -> None:
        self.chapter_name = chapter_name
        self.chapter_text = chapter_text


In [ ]:
class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)

    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)

    def handle_data(self, data):
        print("Encountered some data  :", data)

parser = MyHTMLParser()
parser.feed('&#13;<p class="asangre salto35"><span class="letra">N</span>O llevaba sombrero. Entró con la cabeza echada hacia atrás, como en un desafío. La curva de su nariz recordaba una nave surcando valiente un mar embravecido. En aquel momento, Mary Debenham estaba hermosísima.</p>&#13;&#13;')

Encountered some data  : 
Encountered a start tag: p
Encountered a start tag: span
Encountered some data  : N
Encountered an end tag : span
Encountered some data  : O llevaba sombrero. Entró con la cabeza echada hacia atrás, como en un desafío. La curva de su nariz recordaba una nave surcando valiente un mar embravecido. En aquel momento, Mary Debenham estaba hermosísima.
Encountered an end tag : p
Encountered some data  : 


&#13;
  <h2 class="oculto" title="7. La identidad de Mary Debenham"/>&#13;
&#13;
  <div class="capitulo centrado">&#13;
    <p class="salto30">7</p>&#13;
&#13;
    <p class="salto05">LA IDENTIDAD DE MARY DEBENHAM</p>&#13;
  </div>&#13;
&#13;
  <p class="asangre salto35"><span class="letra">N</span>O llevaba sombrero. Entró con la cabeza echada hacia atrás, como en un desafío. La curva de su nariz recordaba una nave surcando valiente un mar embravecido. En aquel momento, Mary Debenham estaba hermosísima.</p>&#13;
&#13;
  <p>Su mirada se posó en Arbuthnot un instante…, sólo un instante.</p>&#13;
&#13;
  <p>—Deseaba preguntarle, señorita, por qué nos mintió usted esta mañana.</p>&#13;
&#13;
  <p>—¿Mentirle yo? No sé a lo que se refiere.</p>&#13;
&#13;

In [ ]:
for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    print(item)

<EpubHtml:cubierta.xhtml:Text/cubierta.xhtml>
<EpubHtml:sinopsis.xhtml:Text/sinopsis.xhtml>
<EpubHtml:titulo.xhtml:Text/titulo.xhtml>
<EpubHtml:info.xhtml:Text/info.xhtml>
<EpubHtml:dedicatoria.xhtml:Text/dedicatoria.xhtml>
<EpubHtml:part1.xhtml:Text/part1.xhtml>
<EpubHtml:Section0001.xhtml:Text/Section0001.xhtml>
<EpubHtml:Section0002.xhtml:Text/Section0002.xhtml>
<EpubHtml:Section0003.xhtml:Text/Section0003.xhtml>
<EpubHtml:Section0004.xhtml:Text/Section0004.xhtml>
<EpubHtml:Section0005.xhtml:Text/Section0005.xhtml>
<EpubHtml:Section0006.xhtml:Text/Section0006.xhtml>
<EpubHtml:Section0007.xhtml:Text/Section0007.xhtml>
<EpubHtml:Section0008.xhtml:Text/Section0008.xhtml>
<EpubHtml:part2.xhtml:Text/part2.xhtml>
<EpubHtml:Section0009.xhtml:Text/Section0009.xhtml>
<EpubHtml:Section0010.xhtml:Text/Section0010.xhtml>
<EpubHtml:Section0011.xhtml:Text/Section0011.xhtml>
<EpubHtml:Section0012.xhtml:Text/Section0012.xhtml>
<EpubHtml:Section0013.xhtml:Text/Section0013.xhtml>
<EpubHtml:Section001

In [ ]:
# function to get an orderer array of Chapters until a selected chapter
for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    if item.is_chapter():
        bodyContent = item.get_body_content().decode()
        print()
        f = HTMLFilter()
        f.feed(bodyContent)
        content += f.text

NameError: name 'HTMLFilter' is not defined

First we need know which lines are refering to some character or is being talked by them 

PARTE DE TRATAMIENTO

In [ ]:
def search_adj_for_person(person_name,nlp_sentence):
    '''
    return all the adjectives in the sentence that are reffering to the person_name 
    '''
    adjectives = []
    for token in nlp_sentence:

        # If we find a character in the phrase or a NOUN, we get it
        if token.pos_  == "PROPN":
            character = token.text
        
        elif token.pos_  == "NOUN":
            character = None

        
        # If the token is an adjective, and its refering the character we want, we save them
        if character == person_name and token.pos_ == 'ADJ':
            adjectives.append(token.text)

    return adjectives

In [ ]:
# Load the data
text = "Pablo es alto y guapo y tiene una silla bonita, mientras que Juan es bajo y feo y come muy mal"
nlp = spacy.load('es_core_news_md')

# Preprocess the text
doc = nlp(text)

# Initialize containers to store character information
character_info = defaultdict(lambda: {
    "description": [],
    "sentences": [],
    "mentions": 0,
    "relationships": defaultdict(int),
    "actions": []
})

# Iterate through the sentences to extract relevant information
for sent in doc.sents:
    sent_doc = nlp(sent.text)
    characters_in_sentence = []
    
    # We analyze the sentence token by token
    for token in sent_doc:
        # if there is an entity of type person we add all adjectives in the sentence as the description of the entity, until we find another person
        if token.ent_type_ == "PER":
            character_name = token.text
            
            # Add the sentence to the sentences list of that character
            character_info[character_name]["sentences"] += sent.text

            # Add the adjectives to the adjectives list of that character
            character_info[character_name]["description"] += search_adj_for_person(character_name,sent_doc)

            # Add the thigs he migth tell or say to the adjectives list of that character

            if token.dep_ == "ROOT" and token.lemma_ in ["do", "say", "act", "think", "feel"]:
                # Extract actions, e.g., "John ran", "Sarah shouted"
                for char in characters_in_sentence:
                    character_info[char]["actions"].append(sent.text)




print(character_info['Pablo']["description"])
print(character_info['Juan']["description"])

['alto', 'guapo']
['bajo', 'feo']


In [ ]:
agata_text = "-¿Qué sintió al saber lo que había hecho?-preguntó Armstrong-.¿No lo lamentó?¿No se reprochó su conducta? \
    Lombrad exclamó:-¿Yo? No tenía nada que reprocharme."

nlp = spacy.load('es_core_news_md')

# Preprocess the text
doc = nlp(agata_text)

# Initialize containers to store character information
character_info = defaultdict(lambda: {
    "description": [],
    "sentences": [],
    "mentions": 0,
    "relationships": defaultdict(int),
    "actions": []
})

# Iterate through the sentences to extract relevant information
for sent in doc.sents:
    print("----")
    print(sent)
    sent_doc = nlp(sent.text)
    characters_in_sentence = []
    
    # We analyze the sentence token by token
    for token in sent_doc:
        # if there is an entity of type person we add all adjectives in the sentence as the description of the entity, until we find another person
        if token.ent_type_ == "PER":
            character_name = token.text
            
            # Add the sentence to the sentences list of that character
            character_info[character_name]["sentences"] += sent.text

            # Add the adjectives to the adjectives list of that character
            character_info[character_name]["description"] += search_adj_for_person(character_name,sent_doc)



print(character_info)

----
-¿Qué sintió al saber lo que había hecho?-preguntó Armstrong-.¿No lo lamentó?¿No se reprochó su conducta?     
----
Lombrad exclamó:-¿Yo?
----
No tenía nada que reprocharme.
defaultdict(<function <lambda> at 0x0000027CEBB914E0>, {'Lombrad': {'description': [], 'sentences': ['L', 'o', 'm', 'b', 'r', 'a', 'd', ' ', 'e', 'x', 'c', 'l', 'a', 'm', 'ó', ':', '-', '¿', 'Y', 'o', '?'], 'mentions': 0, 'relationships': defaultdict(<class 'int'>, {}), 'actions': []}})


In [ ]:
spacy.explain("ROOT")

'root'

In [ ]:
# Resumir info
from transformers import pipeline

# Cargar un pipeline de resumen
resumir = pipeline('summarization')

# Crear resumenes cortos de las oraciones donde aparece cada personaje
for personaje, oraciones in info_personajes.items():
    print(f"Personaje: {personaje}")
    texto_unido = " ".join(oraciones)
    resumen = resumir(texto_unido[:1000], max_length=100, min_length=30, do_sample=False)
    print(f"Resumen: {resumen[0]['summary_text']}")